In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 43.8 MB/s 
     |████████████████████████████████| 895 kB 42.6 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 6.8 MB 30.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#transformers
from transformers import AdamW,AutoTokenizer
from transformers.optimization import get_cosine_schedule_with_warmup # Lr 스케줄링, 보통 학습 초기엔 높은 Lr로 빠른 탐색 후, 학습 후기 낮은 Lr로 세부 탐색을 진행한다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/My Drive/Colab Notebooks/Data/Korean_Sentence_classification(Dacon)/' # 구글 클라우드 데이터 위치

Mounted at /content/drive


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
train_tmp=pd.read_csv(path+'train_data.csv').set_index('index')
test_tmp=pd.read_csv(path+'test_data.csv').set_index('index')
submission=pd.read_csv(path+'sample_submission.csv')

In [ ]:
tok=AutoTokenizer.from_pretrained("klue/roberta-base")

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
train_tmp['label']=le.fit_transform(train_tmp['label'])
x_test_tmp=test_tmp.iloc[:,:-1]

In [ ]:
y_train_tmp=train_tmp['label']
x_train_tmp=train_tmp.iloc[:,:-1]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x_train_tmp,y_train_tmp,test_size=0.2,stratify=y_train_tmp,shuffle=True,random_state=50)

In [ ]:
def ConvertX(x,tokenizer,max_len):
  sent1=x.iloc[:,0].reset_index(drop=True)
  sent2=x.iloc[:,1].reset_index(drop=True)
  input_ids=[]
  token_type_ids=[]
  attention_mask=[]
  for sen1,sen2 in tqdm(zip(sent1,sent2), total=len(sent1)):
    tmp=tokenizer.encode_plus(
        sen1,sen2,
        max_length=max_len,
        pad_to_max_length=True,
        #padding='max_length',
        )
    input_ids.append(tmp['input_ids'])
    token_type_ids.append(tmp['token_type_ids'])
    attention_mask.append(tmp['attention_mask'])
  input_ids=np.array(input_ids,dtype=int)
  token_type_ids=np.array(token_type_ids,dtype=int)
  attention_mask=np.array(attention_mask,dtype=int)
  return (input_ids,attention_mask,token_type_ids)
    

In [ ]:
# Setting parameters
max_len = 100 # 최대 문장 길이
batch_size = 32
warmup_ratio = 0.2
num_epochs = 15
max_grad_norm = 1
learning_rate =  1e-5

In [ ]:
x_train_convert=ConvertX(x_train,tok,max_len)
x_val_convert=ConvertX(x_val,tok,max_len)
x_test_convert=ConvertX(x_test_tmp,tok,max_len)
y_train=y_train.reset_index(drop=True)
y_val=y_val.reset_index(drop=True)

  0%|          | 0/19998 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 1666/1666 [00:00<00:00, 4213.51it/s]


In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [ ]:
from transformers import TFAutoModel
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import initializers
from tensorflow_addons.optimizers import AdamW
from tensorflow_addons.optimizers import RectifiedAdam
from tensorflow_addons.optimizers import AdaBelief

In [ ]:
class KLUEClassifier(Model):
    def __init__(self, model, hidden_size = 768, num_classes=3, dr_rate=0): #클래스 수 확인
        super(KLUEClassifier, self).__init__() # Classifier가 상속한 Model 인자를 Classifier가 가져온다.
        self.model = model.from_pretrained("klue/roberta-base",num_labels=num_classes,from_pt=True) # model: TFAutoModel
                  
        #tmp_classifier= Sequential()
        #tmp_classifier.add(layers.Dense(input_dim=hidden_size,units=num_classes))
        #tmp_classifier.add(layers.Activation('softmax'))
        #tmp_classifier.add(layers.Dropout(rate=dr_rate))
        #self.classifier=tmp_classifier
        self.classifier=layers.Dense(num_classes,kernel_initializer=initializers.he_uniform(seed=None),activation='softmax')
        # kernel_initializer
        # initializers.TruncatedNormal(0.02)
        # initializers.he_uniform(seed=None)
        # 단층 nn (입력 층 hidden_size, 출력층 num_classes)

    def call(self, inputs): # 모델과 함께 데이터를 입력하면 자동으로 실행되는 함수    
        Out = self.model(input_ids = inputs[0], attention_mask = inputs[1], token_type_ids = inputs[2]) # 모델 실행
        return self.classifier(Out[1])

In [ ]:
t_total = len(y_train)//batch_size * num_epochs
#warmup_step = int(t_total * warmup_ratio) # warmup 방식 -> 학습률을 점차 상승 시켰다가 정점을 지난 후 다시 감소시키는 방식

#step = tf.Variable(0, trainable=False)
#schedule = optimizers.schedules.CosineDecay(learning_rate,warmup_step)

#optimizer와 schedule 설정
#optimizer = AdamW(learning_rate=learning_rate, weight_decay=schedule)
optimizer = RectifiedAdam(learning_rate=learning_rate, total_steps=t_total, warmup_proportion=warmup_ratio,min_lr=1e-8) # Lr은 t_total에서 warmup_ratio 만큼 증가했다가 그 이후로는 감소한다.
#optimizer = AdaBelief(learning_rate=learning_rate, warmup_proportion=warmup_ratio, rectify=True)
loss = 'sparse_categorical_crossentropy' # 다중분류를 위한 손실함수 이 때 sparse는 원핫인코딩을 하지 않아도 되는 손실함수이다

In [ ]:
with tf.device('/device:GPU:0'): # 모델 학습에 있어 GPU 학습 사용
  Mod=KLUEClassifier(TFAutoModel,hidden_size=1024,dr_rate=0)
  Mod.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
  Mod.fit(
      x_train_convert,y_train,
      batch_size=batch_size, 
      epochs=num_epochs,   
      validation_data=(x_val_convert,y_val),
      callbacks=[EarlyStopping(monitor='val_accuracy',patience=10)]
      #verbose=1,
      ) #전 과정을 화면에 출력(1) or 미출력(0)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.decoder.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream 

Epoch 1/20
625/625 [==============================] - 921s 1s/step - loss: 1.0679 - accuracy: 0.3869 - val_loss: 0.6787 - val_accuracy: 0.7196
Epoch 2/20
625/625 [==============================] - 879s 1s/step - loss: 0.4975 - accuracy: 0.8111 - val_loss: 0.3880 - val_accuracy: 0.8602
Epoch 3/20
625/625 [==============================] - 879s 1s/step - loss: 0.3343 - accuracy: 0.8787 - val_loss: 0.3557 - val_accuracy: 0.8748
Epoch 4/20
625/625 [==============================] - 879s 1s/step - loss: 0.2314 - accuracy: 0.9174 - val_loss: 0.3632 - val_accuracy: 0.8744
Epoch 5/20
625/625 [==============================] - 876s 1s/step - loss: 0.1572 - accuracy: 0.9453 - val_loss: 0.4885 - val_accuracy: 0.8628
Epoch 6/20
625/625 [==============================] - 880s 1s/step - loss: 0.1065 - accuracy: 0.9636 - val_loss: 0.4828 - val_accuracy: 0.8782
Epoch 7/20
625/625 [==============================] - 880s 1s/step - loss: 0.0709 - accuracy: 0.9759 - val_loss: 0.5466 - val_accuracy: 0.8756

In [ ]:
preds_tmp=Mod.predict(x_test_convert)
preds=[np.argmax(pred) for pred in preds_tmp]

In [ ]:
label_idx = dict(zip( le.transform(list(le.classes_)),list(le.classes_)))
label_idx

In [ ]:
preds_decode=[label_idx[pred] for pred in preds]

In [ ]:
submission['label'] = preds_decode
submission.to_csv(path+"submission.csv", index = False)